In [ ]:
import pandas as pd
from itertools import product

df = pd.read_csv("all_results.csv")

In [ ]:
from tqdm import tqdm

fissile = ["U233", "U235", "PU239", "PU241"]
df["Total fissile"]= df[fissile].sum(axis=1)

def calculate_limit_burnup(grouped_df):
    if grouped_df["KINF (PCM)"].iloc[0] < 1:
        return -1
    if all(grouped_df["KINF (PCM)"] > 1):
        return 1e20
    
    kinf_below_1 = next(i for i, kinf in enumerate(grouped_df["KINF (PCM)"]) if kinf < 1)
    
    y2 = grouped_df["KINF (PCM)"].iloc[kinf_below_1]
    y1 = grouped_df["KINF (PCM)"].iloc[kinf_below_1-1]
    x2 = grouped_df["BURNUP (Mwd/t)"].iloc[kinf_below_1]
    x1 = grouped_df["BURNUP (Mwd/t)"].iloc[kinf_below_1-1]

    a = (y2 - y1) / (x2 - x1)
    b = y1 - x1 * a
    return (1 - b) / a


def calculate_conversion_ratio_at_burnup(grouped_df, input_burnup):
    idx = next((i for i, burnup in enumerate(grouped_df["BURNUP (Mwd/t)"]) if burnup > input_burnup), len(grouped_df))
    return grouped_df["Total fissile"].iloc[idx-1] / grouped_df["Total fissile"].iloc[0]
    

for keys, grouped_df in tqdm(df.groupby(["Thorium Chain", "U233 Input", "U235 Input", "PUTOT Input", "Moderator radius"])):
    cell_burnup = calculate_limit_burnup(grouped_df)
    if cell_burnup > 0:
        true_burnup = 2 * cell_burnup
        cr = calculate_conversion_ratio_at_burnup(grouped_df, true_burnup)
        if cr > 1 and true_burnup > 4000:
            print(keys, cr, true_burnup)
